In [1]:
import geopandas as gpd
import vaex
import json
from shapely.geometry import shape

In [2]:
# Load Vaex DataFrame with points
df = vaex.open('data/trips/trips.hdf5')


In [15]:
filtered_df = df.select(df['Pickup Centroid Latitude'] == df['Pickup Centroid Latitude'])
#selected_columns_df = df['Pickup Centroid Latitude']
#df_cleaned.info()

AttributeError: 'NoneType' object has no attribute 'type'

column,type,unit,description,expression
Trip ID,str,,,
Taxi ID,str,,,
Trip Start Timestamp,str,,,
Trip End Timestamp,str,,,
Trip Seconds,float64,,,
Trip Miles,float64,,,
Pickup Census Tract,float64,,,
Dropoff Census Tract,float64,,,
Pickup Community Area,float64,,,
Dropoff Community Area,float64,,,


In [ ]:
# Create GeoJSON column from longitude and latitude
def create_geojson(lon, lat):
    if lon is not None and lat is not None:
        return {
            
            #gpd.points_from_xy(df[lon], df[lat])
            'type': 'POINT',
            'coordinates': [float(lon), float(lat)]
        }
    return None

df['Pickup Centroid Location_GEOJSON'] = df.apply(create_geojson, arguments=[df['Pickup Centroid Longitude'], df['Pickup Centroid Latitude']])

# Verify the updated column format
print(df['Pickup Centroid Location_GEOJSON'].dtype)


In [49]:
# Create GeoJSON column from longitude and latitude
def create_geojson(lon, lat):
    if lon is not None and lat is not None:
        geojson_dict = {
            'type': 'Point',
            'coordinates': [float(lon), float(lat)]
        }
        return json.dumps(geojson_dict)
    return None

df['Pickup Centroid Location_GEOJSON'] = df.apply(create_geojson, arguments=[df['Pickup Centroid Longitude'], df['Pickup Centroid Latitude']])

# Verify the updated column format
print(df['Pickup Centroid Location_GEOJSON'].dtype)



string


In [53]:
# Extract Point geometry from GeoJSON column
df['point_geometry'] = df['Pickup Centroid Location_GEOJSON'].apply(lambda x: shape(eval(x)) if x is not None else None)

df['point_geometry']

Expression = point_geometry
Length: 37,395,436 dtype: object (column)
-----------------------------------------
Error evaluating: NameError("name 'NaN' is not defined")

In [ ]:
# Load GeoDataFrame with hexagonal shapes from GeoJSON
gdf = gpd.read_file('data/hexagon_files/hexagon_res7.geojson')

# Load Vaex DataFrame with points
#df = vaex.open('data/trips/trips.hdf5')

# Create a spatial index for faster spatial queries
gdf.sindex

# Define a function to check if a point lies within a hexagon
def is_point_inside_hexagon(point, hexagon):
    return point.any().within(hexagon)

# Create a new column in the Vaex DataFrame to store the join result
df['hexagon_tripstart_res7'] = '-1'  # Initialize with a default value indicating no match

# Iterate over each row in the GeoDataFrame
for index, hexagon in gdf.iterrows():
    # print(hexagon["geometry"])
    # Check if any point in the Vaex DataFrame lies within the hexagon
    points_in_hexagon = df[df['Pickup Centroid Location'].apply(lambda point: is_point_inside_hexagon(point, hexagon))]
    if len(points_in_hexagon) > 0:
        # Set the hexagon ID for the matching points in the Vaex DataFrame
        df.loc[points_in_hexagon.index, 'hexagon_tripstart_res7'] = index

# # Save the updated Vaex DataFrame with the hexagon IDs
# df.export_hdf5('path/to/joined_vaexfile.hdf5')


In [ ]:
# Load Vaex DataFrame
df = vaex.open('data/trips/trips.hdf5')

# Convert the column from string to GeoJSON format
def parse_json(x):
    try:
        return json.loads(x)
    except (json.JSONDecodeError, TypeError):
        return None

df['Pickup Centroid Location'] = df['Pickup Centroid Location'].apply(parse_json)

# Verify the updated column format
print(df['Pickup Centroid Location'].dtype)

# Save the updated Vaex DataFrame
#df.export_hdf5('path/to/updated_vaexfile.hdf5')
